### Libraries:

In [1]:
!pip install sklearn-genetic-opt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.3 MB/s eta 0:00:00


In [2]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.8 MB/s eta 0:00:00


In [3]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn_genetic.space import Categorical, Continuous
from hyperopt import fmin, tpe, hp, Trials
import numpy as np
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
#from my_optimizer import ModelOptimizer

from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn import preprocessing

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

### Warnings:

In [4]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The objective has been evaluated at this point before.")

### Read dataset:

In [5]:
from google.colab import files
uploaded = files.upload()

Saving data2_filled.csv to data2_filled.csv


In [6]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['data2_filled.csv']))
df

,Study_Folder,Subject_Folder,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271,0.124451,0.166400
1,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382,0.124451,0.166400
2,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633,0.124451,0.166400
3,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340,0.124451,0.166400
4,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772,0.124451,0.166400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60420,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,139,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763,0.125000,0.166667
60421,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,140,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050,0.125000,0.166667
60422,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,141,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886,0.125000,0.165266
60423,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,142,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049,0.124349,0.165266


In [ ]:
# import pandas as pd
# df = pd.read_csv('data2_filled.csv')
# df

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60425 entries, 0 to 60424
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Study_Folder    60425 non-null  object 
 1   Subject_Folder  60425 non-null  object 
 2   Stimuli         60425 non-null  int64  
 3   Frame Number    60425 non-null  int64  
 4   Face_Y          60425 non-null  float64
 5   Face_X          60425 non-null  float64
 6   Face_W          60425 non-null  float64
 7   Face_H          60425 non-null  float64
 8   Right_Eye_X     60425 non-null  float64
 9   Right_Eye_Y     60425 non-null  float64
 10  Left_Eye_X      60425 non-null  float64
 11  Left_Eye_Y      60425 non-null  float64
 12  Face_Angle      60425 non-null  float64
 13  X_Predictions   60425 non-null  float64
 14  Y_Predictions   60425 non-null  float64
dtypes: float64(11), int64(2), object(2)
memory usage: 6.9+ MB


### Drop columns:

In [8]:
df.drop(['Study_Folder','Subject_Folder'], axis = 1, inplace = True)

In [9]:
df

,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0,0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271,0.124451,0.166400
1,0,1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382,0.124451,0.166400
2,0,2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633,0.124451,0.166400
3,0,3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340,0.124451,0.166400
4,0,4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772,0.124451,0.166400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60420,7998,139,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763,0.125000,0.166667
60421,7998,140,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050,0.125000,0.166667
60422,7998,141,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886,0.125000,0.165266
60423,7998,142,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049,0.124349,0.165266


In [10]:
# Checking NaN values
df.isna().sum()

Stimuli          0
Frame Number     0
Face_Y           0
Face_X           0
Face_W           0
Face_H           0
Right_Eye_X      0
Right_Eye_Y      0
Left_Eye_X       0
Left_Eye_Y       0
Face_Angle       0
X_Predictions    0
Y_Predictions    0
dtype: int64

### Define Stimuli, X,Y:

In [11]:
df.columns

Index(['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W', 'Face_H',
       'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle',
       'X_Predictions', 'Y_Predictions'],
      dtype='object')

In [12]:
features_columns = ['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W','Face_H',
                    'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle']

target_columns=['X_Predictions', 'Y_Predictions']

In [13]:
X=df[features_columns]

In [14]:
y=df[target_columns]

In [15]:
X_train_valid= X[X['Stimuli']<= 11]

In [16]:
X_test= X[X['Stimuli']> 11]

In [17]:
y_train_valid= y[X['Stimuli']<= 11]

In [18]:
y_test= y[X['Stimuli']> 11]

### Split data:

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.3,stratify =X_train_valid['Stimuli'] ,random_state=42)

In [20]:
print("Whole dataset :",len(df))
print("X Train size", len(X_train))
print("y Train size", len(y_train))
print("X Valid size", len(X_valid))
print("y Valid size", len(y_valid))
print("X Test size", len(X_test))
print("y Test size", len(y_test))

Whole dataset : 60425
X Train size 23720
y Train size 23720
X Valid size 10167
y Valid size 10167
X Test size 26538
y Test size 26538


In [21]:
print("X Train shape", X_train.shape)
print("y Train shape", y_train.shape)
print("X Valid shape", X_valid.shape)
print("y Valid shape", y_valid.shape)
print("X Test shape", X_test.shape)
print("y Test shape", y_test.shape)

X Train shape (23720, 11)
y Train shape (23720, 2)
X Valid shape (10167, 11)
y Valid shape (10167, 2)
X Test shape (26538, 11)
y Test shape (26538, 2)


In [22]:
X_train['Stimuli'].value_counts().sort_values()/len(X_train), X_valid['Stimuli'].value_counts().sort_values()/len(X_valid)

(0     0.080017
 9     0.083347
 5     0.083474
 2     0.083474
 4     0.083516
 1     0.083600
 10    0.083642
 3     0.083642
 6     0.083685
 7     0.083769
 8     0.083853
 11    0.083980
 Name: Stimuli, dtype: float64,
 0     0.079965
 9     0.083309
 5     0.083505
 2     0.083505
 4     0.083505
 6     0.083604
 10    0.083604
 1     0.083604
 3     0.083702
 7     0.083801
 8     0.083899
 11    0.083997
 Name: Stimuli, dtype: float64)

### Drop columns:

In [23]:
drop_column = ['Stimuli','Frame Number']

In [24]:
X.drop(columns = drop_column, inplace=True)

<ipython-input-24-02a2a226598b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns = drop_column, inplace=True)


In [25]:
X

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271
1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382
2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633
3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340
4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772
...,...,...,...,...,...,...,...,...,...
60420,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763
60421,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050
60422,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886
60423,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049


In [26]:
X_train.drop(columns = drop_column, inplace=True)

In [27]:
X_train

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
30626,199.0,183.0,205.0,205.0,266.0,340.0,266.0,259.0,0.353743
10603,225.0,187.0,202.0,202.0,260.0,366.0,259.0,286.0,0.469339
28769,110.0,309.0,199.0,199.0,389.0,242.0,391.0,172.0,0.156968
30149,133.0,242.0,206.0,206.0,331.0,271.0,330.0,202.0,0.206556
16125,239.0,160.0,201.0,201.0,238.0,382.0,231.0,310.0,0.550115
...,...,...,...,...,...,...,...,...,...
28521,153.0,195.0,269.0,269.0,302.0,332.0,306.0,237.0,0.230310
17344,148.0,275.0,209.0,209.0,359.0,295.0,345.0,214.0,0.164995
30525,235.0,160.0,225.0,225.0,255.0,393.0,254.0,311.0,0.642546
26050,254.0,163.0,249.0,249.0,262.0,424.0,268.0,337.0,0.501909


In [28]:
X_valid.drop(columns = drop_column, inplace=True)

In [29]:
X_valid

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
127,217.0,146.0,231.0,231.0,230.0,380.0,237.0,296.0,0.381793
32452,228.0,255.0,179.0,179.0,328.0,347.0,328.0,288.0,0.379762
19084,223.0,158.0,210.0,210.0,232.0,367.0,229.0,287.0,0.608490
16347,245.0,155.0,221.0,221.0,249.0,397.0,246.0,320.0,0.688683
16544,195.0,176.0,215.0,215.0,262.0,342.0,264.0,260.0,0.333173
...,...,...,...,...,...,...,...,...,...
22209,197.0,180.0,210.0,210.0,264.0,337.0,264.0,257.0,0.321964
16792,272.0,122.0,261.0,261.0,225.0,454.0,222.0,363.0,1.236665
19581,271.0,120.0,262.0,262.0,226.0,442.0,224.0,358.0,1.236923
9269,174.0,65.0,283.0,283.0,191.0,355.0,187.0,246.0,0.364568


In [30]:
X_test.drop(columns = drop_column, inplace=True)

<ipython-input-30-7ac6106f35b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns = drop_column, inplace=True)


In [31]:
X_test

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
33887,211.0,159.0,236.0,236.0,244.0,369.0,249.0,281.0,0.362482
33888,211.0,157.0,238.0,238.0,243.0,367.0,249.0,283.0,0.379243
33889,210.0,159.0,235.0,235.0,244.0,369.0,249.0,280.0,0.351363
33890,210.0,158.0,236.0,236.0,244.0,368.0,248.0,282.0,0.357945
33891,209.0,158.0,238.0,238.0,244.0,368.0,249.0,282.0,0.360192
...,...,...,...,...,...,...,...,...,...
60420,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763
60421,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050
60422,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886
60423,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049


### MinMaxScaler:

In [32]:
minmax_scaler = preprocessing.MinMaxScaler()
X_train_scal = minmax_scaler.fit_transform(X_train)
X_valid_scal = minmax_scaler.transform(X_valid)
X_test_scal = minmax_scaler.transform(X_test)

In [33]:
X_train_scal

array([[0.36533958, 0.40566038, 0.23668639, ..., 0.38970588, 0.39111111,
        0.00370046],
       [0.42622951, 0.41509434, 0.21893491, ..., 0.37254902, 0.45111111,
        0.00538576],
       [0.15690867, 0.70283019, 0.20118343, ..., 0.69607843, 0.19777778,
        0.00083163],
       ...,
       [0.44964871, 0.35141509, 0.35502959, ..., 0.36029412, 0.50666667,
        0.00791098],
       [0.4941452 , 0.35849057, 0.49704142, ..., 0.39460784, 0.56444444,
        0.00586059],
       [0.264637  , 0.01415094, 0.6035503 , ..., 0.02205882, 0.34666667,
        0.00992458]])

In [34]:
X_valid_scal

array([[0.40749415, 0.31839623, 0.39053254, ..., 0.31862745, 0.47333333,
        0.0041094 ],
       [0.43325527, 0.5754717 , 0.08284024, ..., 0.54166667, 0.45555556,
        0.00407979],
       [0.42154567, 0.34669811, 0.26627219, ..., 0.29901961, 0.45333333,
        0.00741446],
       ...,
       [0.53395785, 0.25707547, 0.5739645 , ..., 0.28676471, 0.61111111,
        0.01657654],
       [0.30679157, 0.12735849, 0.69822485, ..., 0.19607843, 0.36222222,
        0.00385827],
       [0.36299766, 0.45518868, 0.10059172, ..., 0.41176471, 0.39777778,
        0.00418931]])

In [35]:
X_test_scal

array([[0.39344262, 0.3490566 , 0.42011834, ..., 0.34803922, 0.44      ,
        0.00382786],
       [0.39344262, 0.34433962, 0.43195266, ..., 0.34803922, 0.44444444,
        0.00407222],
       [0.3911007 , 0.3490566 , 0.41420118, ..., 0.34803922, 0.43777778,
        0.00366576],
       ...,
       [0.62295082, 0.22641509, 0.35502959, ..., 0.2254902 , 0.67333333,
        0.02678145],
       [0.62529274, 0.22641509, 0.34911243, ..., 0.2254902 , 0.66222222,
        0.03251346],
       [0.6206089 , 0.22641509, 0.35502959, ..., 0.22303922, 0.67111111,
        0.03118879]])

### Save csv file (Train,valid,test):

In [36]:
pd.DataFrame(X_train_scal).to_csv('X_train_scal_MMS2.csv', index=False)

In [38]:
pd.DataFrame(X_valid_scal).to_csv('X_valid_scal_MMS2.csv', index=False)

In [40]:
pd.DataFrame(X_test_scal).to_csv('X_test_scal_MMS2.csv', index=False)

In [42]:
pd.DataFrame(y_train).to_csv('y_train2.csv', index=False)

In [45]:
pd.DataFrame(y_valid).to_csv('y_valid2.csv', index=False)

In [47]:
pd.DataFrame(y_test).to_csv('y_test2.csv', index=False)

In [48]:
from google.colab import files

files.download('y_test2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Function for Model Optimizer:

In [49]:
class ModelOptimizer:
    results = []

    def __init__(self, model, optimization_method='grid', optimization_params=None):
        self.model = model
        self.optimization_method = optimization_method
        self.optimization_params = optimization_params

    def optimize(self, X, y):
        if self.optimization_method == 'grid':
            return self.grid_search(X, y)
        elif self.optimization_method == 'randomized':
            return self.random_search(X, y)
        elif self.optimization_method == 'bayesian':
            return self.bayesian_optimization(X, y)
        elif self.optimization_method == 'bayesian_hyperparameters':
            return self.bayesian_optimization_hyperparameters(X, y)
        elif self.optimization_method == 'genetic':
            return self.genetic_algorithm(X, y)
        else:
            raise ValueError('Invalid optimization method.')

    def grid_search(self, X, y):
        optimizer = GridSearchCV(self.model, self.optimization_params, cv=5)
        optimizer.fit(X, y)
        best_estimator = optimizer.best_estimator_
        best_params = optimizer.best_params_
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def random_search(self, X, y):
        optimizer = RandomizedSearchCV(self.model, self.optimization_params, cv=5)
        optimizer.fit(X, y)
        best_estimator = optimizer.best_estimator_
        best_params = optimizer.best_params_
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def bayesian_optimization(self, X, y):
        optimizer = BayesSearchCV(self.model, self.optimization_params, cv=5)
        optimizer.fit(X, y)
        best_estimator = optimizer.best_estimator_
        best_params = optimizer.best_params_
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def bayesian_optimization_hyperparameters(self, X, y):
        space = self._convert_to_hyperopt_space(self.optimization_params)
        objective = self._create_objective_function(X, y)
        trials = Trials()
        best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)
        best_params = self._convert_hyperopt_params(best)
        best_estimator = self.model.set_params(**best_params)
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def genetic_algorithm(self, X, y):
        ga = GeneticAlgorithm(self.model, self.optimization_params)
        best_estimator, best_params = ga.optimize(X, y)
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def _convert_to_hyperopt_space(self, optimization_params):
        space = {}
        for param, values in optimization_params.items():
            if isinstance(values, list):
                space[param] = hp.choice(param, values)
            else:
                space[param] = values
        return space

    def _create_objective_function(self, X, y):
        def objective(params):
            estimator = self.model.set_params(**params)
            score = -1.0 * np.mean(cross_val_score(estimator, X, y, cv=5, scoring='accuracy'))
            return score

        return objective

    def _convert_hyperopt_params(self, hyperopt_params):
        converted_params = {}
        for param, value in hyperopt_params.items():
            if isinstance(value, int):
                converted_params[param] = self.optimization_params[param][value]
            else:
                converted_params[param] = value
        return converted_params

    def _convert_genetic_params(self, genetic_params):
        converted_params = {}
        for param, value in genetic_params.items():
            if isinstance(value, int):
                converted_params[param] = self.optimization_params[param][value]
            else:
                converted_params[param] = value
        return converted_params

    @classmethod
    def display_results(cls):
        for result in cls.results:
            best_estimator, best_params = result
            print("Best Estimator:", best_estimator)
            print("Best Parameters:", best_params)
            print("---------------")

### Exampe for Decision Tree Regressor:

In [50]:
regressor_model = DecisionTreeRegressor()

# Define the optimization parameters
optimization_method = 'randomized'
optimization_params = {'max_depth': [None, 5, 10],
                       'min_samples_split': [2, 4],
                       'min_samples_leaf':[1],
                       'max_features':[None],
                       'random_state':[42]}

# Create an instance of ModelOptimizer
optimizer = ModelOptimizer(regressor_model, optimization_method, optimization_params)

# Perform the optimization
best_model, best_params = optimizer.optimize(X_train_scal, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [51]:
ModelOptimizer.display_results()

Best Estimator: DecisionTreeRegressor(min_samples_split=4, random_state=42)
Best Parameters: {'random_state': 42, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': None}
---------------


In [52]:
best_model

DecisionTreeRegressor(min_samples_split=4, random_state=42)

In [53]:
best_params

{'random_state': 42,
 'min_samples_split': 4,
 'min_samples_leaf': 1,
 'max_features': None,
 'max_depth': None}

In [54]:
best_model.fit(X_train_scal, y_train)
y_pred = best_model.predict(X_valid_scal)
mean_absolute_error(y_valid, y_pred)

0.04592160311598321

In [55]:
metric_dict = {"r2":r2_score,"mse":mean_squared_error,"mae":mean_absolute_error}

In [56]:
for metric in metric_dict:
    print(metric, metric_dict[metric](y_valid,y_pred))

r2 0.7762758178278858
mse 0.017106522380855655
mae 0.04592160311598321


In [57]:
b_test = best_model.predict(X_test_scal)

In [58]:
for metric in metric_dict:
    print(metric, metric_dict[metric](y_test,b_test))

r2 -0.18371091687144925
mse 0.0890915147698843
mae 0.19746490130510624
